## What I did
In this annotated notebook, I added one-sentence explanations for each section. Some of them were added as comments in the code, and some of them had more than one sentence for me to self-teach the concepts. I also copied, pasted, and modified some code snippets to see their outputs.

## Step 0: Setup
Install TransformerLens so that we can load GPT-2 and inspect internal activations.

In [2]:
# Install TransformerLens into the Colab environment
!pip install transformer_lens

### Plotting function
A helper to visualize the tensors (high-dimensional data, e.g. attention) as heatmaps with labels.

In [3]:
import plotly.express as px

def imshow(
    tensor,
    xlabel="X",
    ylabel="Y",
    zlabel=None,
    xticks=None,
    yticks=None,
    c_midpoint=0.0,
    c_scale="RdBu",
    show=True,
    **kwargs
):
    # Convert tensor to NumPy for Plotly
    tensor = utils.to_numpy(tensor)
    xticks = [str(x) for x in xticks]
    yticks = [str(y) for y in yticks]
    labels = {"x": xlabel, "y": ylabel}
    if zlabel is not None:
        labels["color"] = zlabel
    fig = px.imshow(
        tensor,
        x=xticks,
        y=yticks,
        labels=labels,
        color_continuous_midpoint=c_midpoint,
        color_continuous_scale=c_scale,
        **kwargs
    ).show()

## Note on Errors:

* If your notebook crashes due to CUDA out of memory try (a) Restart notebook, and (b) Go to Runtime, Change Runtime type, select no hardware accelerator (no GPU). Colab without GPU should have enough memory to run the examples shown here

## Step 1: Getting a model to play with

In [4]:
"""
Load GPT-2 small with hooks (access points inside the transformer where we can
inspect and optionally modify internal tensors) support.
"""
from transformer_lens import HookedTransformer, utils

model_gpt = HookedTransformer.from_pretrained("gpt2-small")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loaded pretrained model gpt2-small into HookedTransformer


### Next token prediction
Select the token with highest logit from the final position ([0, -1]) to predict the next token and decode the token into readable text.

In [5]:
logits = model_gpt("Famous computer scientist Alan")
# The logit dimensions are: [batch, position, vocab]
next_token_logits = logits[0, -1]
next_token_prediction = next_token_logits.argmax()
next_word_prediction = model_gpt.tokenizer.decode(next_token_prediction)
print(next_word_prediction)

 Turing


### Access internal activations with `run_with_cache`
Run a forward pass and save intermediate tensors from the hook points, including residual streams and attention patterns, etc., into a cache dictionary.

#### Layer 5 token embedding

In [36]:
test_text = "This is a simple sentence for trying out getting token's embedding at layer 5."
_, test_cache = model_gpt.run_with_cache(test_text)

tokens = model_gpt.to_str_tokens(test_text)
print(tokens)

# The residual stream dimensions are [batch, position, d_embed]
# index the cache dictionary using the hook for that layer
# note that layers are 0-indexed -> blocks.4 corresponds to layer 5
embedding_layer_5 = test_cache["blocks.4.hook_resid_post"][0, -1, :]
embedding_layer_5_resid = test_cache["resid_post", 4][0, -1, :]


print("Shape of embedding at layer 5 (method 1):", embedding_layer_5.shape, "\n")

print("Shape of embedding at layer 5 (method 2):", embedding_layer_5_resid.shape, "\n")

print("Some values (method 1):", embedding_layer_5[:10])
print("Some values (method 2):", embedding_layer_5_resid[:10])

['<|endoftext|>', 'This', ' is', ' a', ' simple', ' sentence', ' for', ' trying', ' out', ' getting', ' token', "'s", ' embed', 'ding', ' at', ' layer', ' 5', '.']
Shape of embedding at layer 5 (method 1): torch.Size([768]) 

Shape of embedding at layer 5 (method 2): torch.Size([768]) 

Some values (method 1): tensor([ 0.9870, -0.7025, -0.2852,  0.7551,  0.5280, -0.1467,  4.2911,  1.5818,
        -2.8366,  0.3248])
Some values (method 2): tensor([ 0.9870, -0.7025, -0.2852,  0.7551,  0.5280, -0.1467,  4.2911,  1.5818,
        -2.8366,  0.3248])


In [8]:
logits, cache = model_gpt.run_with_cache("Famous computer scientist Alan")
for key, value in cache.items():
    print(key, value.shape)

hook_embed torch.Size([1, 6, 768])
hook_pos_embed torch.Size([1, 6, 768])
blocks.0.hook_resid_pre torch.Size([1, 6, 768])
blocks.0.ln1.hook_scale torch.Size([1, 6, 1])
blocks.0.ln1.hook_normalized torch.Size([1, 6, 768])
blocks.0.attn.hook_q torch.Size([1, 6, 12, 64])
blocks.0.attn.hook_k torch.Size([1, 6, 12, 64])
blocks.0.attn.hook_v torch.Size([1, 6, 12, 64])
blocks.0.attn.hook_attn_scores torch.Size([1, 12, 6, 6])
blocks.0.attn.hook_pattern torch.Size([1, 12, 6, 6])
blocks.0.attn.hook_z torch.Size([1, 6, 12, 64])
blocks.0.hook_attn_out torch.Size([1, 6, 768])
blocks.0.hook_resid_mid torch.Size([1, 6, 768])
blocks.0.ln2.hook_scale torch.Size([1, 6, 1])
blocks.0.ln2.hook_normalized torch.Size([1, 6, 768])
blocks.0.mlp.hook_pre torch.Size([1, 6, 3072])
blocks.0.mlp.hook_post torch.Size([1, 6, 3072])
blocks.0.hook_mlp_out torch.Size([1, 6, 768])
blocks.0.hook_resid_post torch.Size([1, 6, 768])
blocks.1.hook_resid_pre torch.Size([1, 6, 768])
blocks.1.ln1.hook_scale torch.Size([1, 6, 1])

### Access all the weights and parameters of the model with `named_parameters`
Get the weight matrices (Query vector, Key vector, and Value vector) and biases (learned numeric offset) of every MLP and attention layer.

In [9]:
for name, parameter in model_gpt.named_parameters():
    print(name, parameter.shape)

embed.W_E torch.Size([50257, 768])
pos_embed.W_pos torch.Size([1024, 768])
blocks.0.attn.W_Q torch.Size([12, 768, 64])
blocks.0.attn.W_O torch.Size([12, 64, 768])
blocks.0.attn.b_Q torch.Size([12, 64])
blocks.0.attn.b_O torch.Size([768])
blocks.0.attn.W_K torch.Size([12, 768, 64])
blocks.0.attn.W_V torch.Size([12, 768, 64])
blocks.0.attn.b_K torch.Size([12, 64])
blocks.0.attn.b_V torch.Size([12, 64])
blocks.0.mlp.W_in torch.Size([768, 3072])
blocks.0.mlp.b_in torch.Size([3072])
blocks.0.mlp.W_out torch.Size([3072, 768])
blocks.0.mlp.b_out torch.Size([768])
blocks.1.attn.W_Q torch.Size([12, 768, 64])
blocks.1.attn.W_O torch.Size([12, 64, 768])
blocks.1.attn.b_Q torch.Size([12, 64])
blocks.1.attn.b_O torch.Size([768])
blocks.1.attn.W_K torch.Size([12, 768, 64])
blocks.1.attn.W_V torch.Size([12, 768, 64])
blocks.1.attn.b_K torch.Size([12, 64])
blocks.1.attn.b_V torch.Size([12, 64])
blocks.1.mlp.W_in torch.Size([768, 3072])
blocks.1.mlp.b_in torch.Size([3072])
blocks.1.mlp.W_out torch.Size

### Test
Check how the model predicts the specified target token by comparing its rank and probability among next-token predictions.

In [10]:
utils.test_prompt("Her name was Alex Hart. Tomorrow at lunch time Alex",
                  " Hart", model_gpt)

Tokenized prompt: ['<|endoftext|>', 'Her', ' name', ' was', ' Alex', ' Hart', '.', ' Tomorrow', ' at', ' lunch', ' time', ' Alex']
Tokenized answer: [' Hart']


Performance on answer token:
Rank: 3        Logit: 14.29 Prob:  7.35% Token: | Hart|

Top 0th token. Logit: 15.64 Prob: 28.38% Token: | will|
Top 1th token. Logit: 14.47 Prob:  8.79% Token: | would|
Top 2th token. Logit: 14.34 Prob:  7.74% Token: | was|
Top 3th token. Logit: 14.29 Prob:  7.35% Token: | Hart|
Top 4th token. Logit: 14.18 Prob:  6.54% Token: | and|
Top 5th token. Logit: 14.09 Prob:  6.00% Token: | is|
Top 6th token. Logit: 13.51 Prob:  3.38% Token: |'s|
Top 7th token. Logit: 13.23 Prob:  2.53% Token: |,|
Top 8th token. Logit: 12.73 Prob:  1.55% Token: | had|
Top 9th token. Logit: 12.00 Prob:  0.74% Token: | has|


Ranks of the answer tokens: [(' Hart', 3)]

### Method 1: Residual Stream Patching
Replace the vector at token position `pos` with the value from the corrupted run and observe any changes in prediction.

#### My introductory explanation

In [11]:
model_gpt.reset_hooks()
_, corrupt_cache_gpt = model_gpt.run_with_cache("Her name was Alex Carroll. Tomorrow at lunch time Alex")

def patch_residual_stream(activations, hook, layer="blocks.2.hook_resid_post", pos=5):
   # The residual stream dimensions are [batch, position, d_embed]
   activations[:, pos, :] = corrupt_cache_gpt[layer][:, pos, :]
   return activations


# add_hook takes 2 args: Where to insert the patch,
# and the function providing the updated activations
model_gpt.add_hook("blocks.2.hook_resid_post", patch_residual_stream)


utils.test_prompt("Her name was Alex Hart. Tomorrow at lunch time Alex", " Hart", model_gpt)

model_gpt.reset_hooks()


Tokenized prompt: ['<|endoftext|>', 'Her', ' name', ' was', ' Alex', ' Hart', '.', ' Tomorrow', ' at', ' lunch', ' time', ' Alex']
Tokenized answer: [' Hart']


Performance on answer token:
Rank: 2882     Logit:  4.81 Prob:  0.00% Token: | Hart|

Top 0th token. Logit: 15.63 Prob: 30.45% Token: | will|
Top 1th token. Logit: 14.44 Prob:  9.23% Token: | was|
Top 2th token. Logit: 14.42 Prob:  9.11% Token: | Carroll|
Top 3th token. Logit: 14.17 Prob:  7.04% Token: | is|
Top 4th token. Logit: 13.88 Prob:  5.31% Token: | and|
Top 5th token. Logit: 13.76 Prob:  4.70% Token: | would|
Top 6th token. Logit: 13.38 Prob:  3.20% Token: |'s|
Top 7th token. Logit: 12.93 Prob:  2.04% Token: |,|
Top 8th token. Logit: 12.33 Prob:  1.12% Token: | had|
Top 9th token. Logit: 12.01 Prob:  0.81% Token: | has|


Ranks of the answer tokens: [(' Hart', 2882)]

### Steven's attempt to method 1

In [12]:
print("Steven's sentence:\n")

print("Before corrupt:\n")
utils.test_prompt("Our team members are Mario and Steven. Tommorrow during the meeting Mario and",
                  " Steven", model_gpt)
print("")
print("After corrupt:\n")
model_gpt.reset_hooks()
_, corrupt_cache_gpt = model_gpt.run_with_cache("Our team members are Mario and Gabrielle. Tommorrow during the meeting Mario and")

def patch_residual_stream(activations, hook, layer="blocks.2.hook_resid_post", pos=7):
   # The residual stream dimensions are [batch, position, d_embed]
   activations[:, pos, :] = corrupt_cache_gpt[layer][:, pos, :]
   return activations


# add_hook takes 2 args: Where to insert the patch,
# and the function providing the updated activations
model_gpt.add_hook("blocks.2.hook_resid_post", patch_residual_stream)


utils.test_prompt("Our team members are Mario and Steven. Tommorrow during the meeting Mario and",
                  " Steven", model_gpt)

model_gpt.reset_hooks()





Steven's sentence:

Before corrupt:

Tokenized prompt: ['<|endoftext|>', 'Our', ' team', ' members', ' are', ' Mario', ' and', ' Steven', '.', ' T', 'omm', 'orrow', ' during', ' the', ' meeting', ' Mario', ' and']
Tokenized answer: [' Steven']


Performance on answer token:
Rank: 0        Logit: 15.26 Prob: 52.44% Token: | Steven|

Top 0th token. Logit: 15.26 Prob: 52.44% Token: | Steven|
Top 1th token. Logit: 12.90 Prob:  4.97% Token: | Steve|
Top 2th token. Logit: 12.83 Prob:  4.62% Token: | I|
Top 3th token. Logit: 12.72 Prob:  4.15% Token: | his|
Top 4th token. Logit: 12.11 Prob:  2.26% Token: | the|
Top 5th token. Logit: 12.00 Prob:  2.02% Token: | T|
Top 6th token. Logit: 11.08 Prob:  0.80% Token: | we|
Top 7th token. Logit: 10.73 Prob:  0.57% Token: | he|
Top 8th token. Logit: 10.69 Prob:  0.55% Token: | our|
Top 9th token. Logit: 10.46 Prob:  0.43% Token: | Mario|


Ranks of the answer tokens: [(' Steven', 0)]


After corrupt:

Tokenized prompt: ['<|endoftext|>', 'Our', ' team', ' members', ' are', ' Mario', ' and', ' Steven', '.', ' T', 'omm', 'orrow', ' during', ' the', ' meeting', ' Mario', ' and']
Tokenized answer: [' Steven']


Performance on answer token:
Rank: 514      Logit:  6.73 Prob:  0.01% Token: | Steven|

Top 0th token. Logit: 15.86 Prob: 65.40% Token: | Gabriel|
Top 1th token. Logit: 13.09 Prob:  4.11% Token: | I|
Top 2th token. Logit: 12.56 Prob:  2.42% Token: | his|
Top 3th token. Logit: 12.32 Prob:  1.89% Token: | Gabe|
Top 4th token. Logit: 12.08 Prob:  1.48% Token: | the|
Top 5th token. Logit: 11.75 Prob:  1.07% Token: | Gab|
Top 6th token. Logit: 11.27 Prob:  0.66% Token: | Mario|
Top 7th token. Logit: 11.26 Prob:  0.65% Token: | Luigi|
Top 8th token. Logit: 11.16 Prob:  0.60% Token: | T|
Top 9th token. Logit: 10.81 Prob:  0.42% Token: | G|


Ranks of the answer tokens: [(' Steven', 514)]

#### How I actually use this

Oops Colab doesn't have enough memory for doing this on GPT2, so use this 6 layer model from Neel. In the LW post I use GPT2-small.


```
model = HookedTransformer.from_pretrained("solu-6l")
```

#### More "automatic" approach

- Loop over different layers and token positions
- Patch the residual stream from a corrupted run to see where the model stores information that affects the final token prediction.
- ***Residual stream:*** the main vector representation of each token that flows through every transformer layer, accumulating information from, for instance, attention and MLP updates.
  *   A not that good analogy is probably to think residual stream as a shared white board, where each intermediate part writes/adds a small modification while the infomation stays and keeps flowing to the end.

In [13]:
import torch
from functools import partial

model_6l = HookedTransformer.from_pretrained("solu-6l")


#corrupt_prompt = "Her name was Sarah Hart. Tomorrow at lunch time Alex"
#corrupt_prompt = "Her name was Alex Hart. Tomorrow at lunch time Sarah"

# Clean and corrupt prompts in variables
clean_prompt = "Her name was Alex Hart. Tomorrow at lunch time Alex"
corrupt_prompt = "Her name was Alex Carroll. Tomorrow at lunch time Alex"
# Get the list of tokens the model will deal with
clean_tokens = model_6l.to_str_tokens(clean_prompt)
# Indices of the right and wrong answers (last names) to judge what the model predicts
_, corrupt_cache_6l = model_6l.run_with_cache(corrupt_prompt)

def patch_residual_stream(activations, hook, layer="blocks.6.hook_resid_post", pos=5):
   # The residual stream dimensions are [batch, position, d_embed]
   activations[:, pos, :] = corrupt_cache_6l[layer][:, pos, :]
   return activations

# List of layers and positions to iterate over. We want to patch before the
# first layer, and after every layer (so we cover 13 positions in total).
layers = ["blocks.0.hook_resid_pre", *[f"blocks.{i}.hook_resid_post" for i in range(model_6l.cfg.n_layers)]]
n_layers = len(layers)
n_pos = len(clean_tokens)

# Indices of the right and wrong answers (last names) to judge what the model predicts
clean_answer_index = model_6l.tokenizer.encode(" Hart")[0]
corrupt_answer_index = model_6l.tokenizer.encode(" Carroll")[0]

# Test the effect of this patch at any layer and any position
patching_effect = torch.zeros(n_layers, n_pos)
for l, layer in enumerate(layers):
    print("Patching layer", l)
    for pos in range(n_pos):
        # Create a hook that patches the residual stream at this layer and token position.
        fwd_hooks = [(layer, partial(patch_residual_stream, layer=layer, pos=pos))]
        prediction_logits = model_6l.run_with_hooks(clean_prompt,
                                                    fwd_hooks=fwd_hooks)[0, -1]
        # Logits difference
        patching_effect[l, pos] = prediction_logits[clean_answer_index] \
                                  - prediction_logits[corrupt_answer_index]

torch.cuda.empty_cache()

Loaded pretrained model solu-6l into HookedTransformer
Patching layer 0
Patching layer 1
Patching layer 2
Patching layer 3
Patching layer 4
Patching layer 5
Patching layer 6


In [14]:
# Plot
token_labels = [f"(pos {i:2}) {t}" for i, t in enumerate(clean_tokens)]
imshow(patching_effect, xticks=token_labels, yticks=layers, xlabel="pos", ylabel="layer",
       zlabel="Logit difference", title="Patching with 1st occurrence of first name", width=600, height=380)


### Steven's attempt

In [15]:
import torch
from functools import partial

model_6l = HookedTransformer.from_pretrained("solu-6l")

# Clean and corrupt prompts in variables
clean_prompt = "Our team members are Mario and Steven. Tommorrow during the meeting Mario and"
corrupt_prompt = "Our team members are Mario and Polly. Tommorrow during the meeting Mario and"
# Get the list of tokens the model will deal with
clean_tokens = model_6l.to_str_tokens(clean_prompt)
# Indices of the right and wrong answers (last names) to judge what the model predicts
_, corrupt_cache_6l = model_6l.run_with_cache(corrupt_prompt)

def patch_residual_stream(activations, hook, layer="blocks.6.hook_resid_post", pos=7):
   # The residual stream dimensions are [batch, position, d_embed]
   activations[:, pos, :] = corrupt_cache_6l[layer][:, pos, :]
   return activations

# List of layers and positions to iterate over. We want to patch before the
# first layer, and after every layer (so we cover 13 positions in total).
layers = ["blocks.0.hook_resid_pre", *[f"blocks.{i}.hook_resid_post" for i in range(model_6l.cfg.n_layers)]]
n_layers = len(layers)
n_pos = len(clean_tokens)

# Indices of the right and wrong answers (last names) to judge what the model predicts
clean_answer_index = model_6l.tokenizer.encode(" Steven")[0]
corrupt_answer_index = model_6l.tokenizer.encode(" Polly")[0]

# Test the effect of this patch at any layer and any position
patching_effect = torch.zeros(n_layers, n_pos)
for l, layer in enumerate(layers):
    print("Patching layer", l)
    for pos in range(n_pos):
        # Create a hook that patches the residual stream at this layer and token position.
        fwd_hooks = [(layer, partial(patch_residual_stream, layer=layer, pos=pos))]
        prediction_logits = model_6l.run_with_hooks(clean_prompt,
                                                    fwd_hooks=fwd_hooks)[0, -1]
        # Logits difference
        patching_effect[l, pos] = prediction_logits[clean_answer_index] \
                                  - prediction_logits[corrupt_answer_index]

torch.cuda.empty_cache()

Loaded pretrained model solu-6l into HookedTransformer
Patching layer 0
Patching layer 1
Patching layer 2
Patching layer 3
Patching layer 4
Patching layer 5
Patching layer 6


In [16]:
# Plot
token_labels = [f"(pos {i:2}) {t}" for i, t in enumerate(clean_tokens)]
imshow(patching_effect, xticks=token_labels, yticks=layers, xlabel="pos", ylabel="layer",
       zlabel="Logit difference", title="Patching with different name", width=600, height=380)


##### Plots of 2 layer model, as used in the post:
Same as previous section but with a different model.

In [17]:
model_2l = HookedTransformer.from_pretrained("gelu-2l")

clean_prompt = "Her name was Alex Hart. Tomorrow at lunch time Alex"
clean_tokens = model_2l.to_str_tokens(clean_prompt)

# Indices of the right and wrong answers (lastnames) to judge what the model predicts
clean_answer_index = model_2l.tokenizer.encode(" Hart")[0]
corrupt_answer_index = model_2l.tokenizer.encode(" Carroll")[0]

def patch_residual_stream(activations, hook, layer="blocks.6.hook_resid_post", pos=5):
   # The residual stream dimensions are [batch, position, d_embed]
   activations[:, pos, :] = corrupt_cache[layer][:, pos, :]
   return activations

layers = ["blocks.0.hook_resid_pre", *[f"blocks.{i}.hook_resid_post" for i in range(model_2l.cfg.n_layers)]]
n_layers = len(layers)
n_pos = len(clean_tokens)


for corrupt_prompt in ["Her name was Alex Carroll. Tomorrow at lunch time Alex",
                       "Her name was Sarah Hart. Tomorrow at lunch time Alex",
                       "Her name was Alex Hart. Tomorrow at lunch time Sarah"]:
    corrupt_tokens = model_2l.to_str_tokens(corrupt_prompt)
    _, corrupt_cache = model_2l.run_with_cache(corrupt_prompt)

    # Test this patch at any layer and any position
    patching_effect = torch.zeros(n_layers, n_pos)
    for l, layer in enumerate(layers):
        for pos in range(n_pos):
            torch.cuda.empty_cache()
            fwd_hooks = [(layer, partial(patch_residual_stream, layer=layer, pos=pos))]
            prediction_logits = model_2l.run_with_hooks(clean_prompt, fwd_hooks=fwd_hooks)[0, -1]
            patching_effect[l, pos] = prediction_logits[clean_answer_index] - prediction_logits[corrupt_answer_index]

    # Plot
    token_labels = [f"(pos {i:2}) {t}" for i, t in enumerate(clean_tokens)]
    imshow(patching_effect, xticks=token_labels, yticks=layers, xlabel="pos", ylabel="layer",
           zlabel="logit diff", title=f"Patching with{corrupt_tokens[4]} /{corrupt_tokens[5]} /{corrupt_tokens[11]}", width=600, height=380)


Loaded pretrained model gelu-2l into HookedTransformer


# Method 2
Unlike method 1 (patching the entire residual stream), method 2 patches the result of one attention head with values from a corrupted run, then again observing how the logits change between the two and thus identifying which head matters.

In [18]:
"""
Set to True to let TransformerLens know that we want to store and expose
each attention head's output individually.
"""
model_2l.cfg.use_attn_result = True


In [19]:
clean_prompt = "Her name was Alex Hart. Tomorrow at lunch time Alex"
#corrupt_prompt = "Her name was Sarah Hart. Tomorrow at lunch time Alex"
#corrupt_prompt = "Her name was Alex Hart. Tomorrow at lunch time Sarah"
corrupt_prompt = "Her name was Alex Carroll. Tomorrow at lunch time Alex"
clean_tokens = model_2l.to_str_tokens(clean_prompt)
_, corrupt_cache = model_2l.run_with_cache(corrupt_prompt)

clean_answer_index = model_2l.tokenizer.encode(" Hart")[0]
corrupt_answer_index = model_2l.tokenizer.encode(" Carroll")[0]

n_layers = model_2l.cfg.n_layers
n_heads = model_2l.cfg.n_heads
n_pos = len(clean_tokens)

def patch_head_result(activations, hook, layer=None, head=None):
   activations[:, :, head, :] = corrupt_cache[hook.name][:, :, head, :]
   return activations

patching_effect = torch.zeros(n_layers, n_heads)
for layer in range(n_layers):
    for head in range(n_heads):
       fwd_hooks = [(f"blocks.{layer}.attn.hook_result", partial(patch_head_result, layer=layer, head=head))]
       prediction_logits = model_2l.run_with_hooks(clean_prompt, fwd_hooks=fwd_hooks)[0, -1]
       patching_effect[layer, head] = prediction_logits[clean_answer_index] - prediction_logits[corrupt_answer_index]

In [20]:
# Plot
token_labels = [f"(pos {i:2}) {t}" for i, t in enumerate(clean_tokens)]
head_labels = [f"(head {i:2})" for i in range(n_heads)]
layer_labels = [f"(layer {i:2})" for i in range(n_layers)]

imshow(patching_effect, xticks=head_labels, yticks=layer_labels, xlabel="head", ylabel="layer",
       zlabel="Logit difference", title="Patching with 1st occurrence of first name", width=600, height=380)


In [21]:
model_2l.cfg.use_attn_result = True


clean_prompt = "Her name was Alex Hart. Tomorrow at lunch time Alex"
clean_tokens = model_2l.to_str_tokens(clean_prompt)

# Indices of the right and wrong answers (lastnames) to judge what the model predicts
clean_answer_index = model_2l.tokenizer.encode(" Hart")[0]
corrupt_answer_index = model_2l.tokenizer.encode(" Carroll")[0]


n_layers = model_2l.cfg.n_layers
n_heads = model_2l.cfg.n_heads
n_pos = len(clean_tokens)

def patch_head_result(activations, hook, layer=None, head=None):
   activations[:, :, head, :] = corrupt_cache[hook.name][:, :, head, :]
   return activations



for corrupt_prompt in ["Her name was Alex Carroll. Tomorrow at lunch time Alex",
                       "Her name was Sarah Hart. Tomorrow at lunch time Alex",
                       "Her name was Alex Hart. Tomorrow at lunch time Sarah"]:
    corrupt_tokens = model_2l.to_str_tokens(corrupt_prompt)
    _, corrupt_cache = model_2l.run_with_cache(corrupt_prompt)

    patching_effect = torch.zeros(n_layers, n_heads)
    for layer in range(n_layers):
        for head in range(n_heads):
           fwd_hooks = [(f"blocks.{layer}.attn.hook_result", partial(patch_head_result, layer=layer, head=head))]
           prediction_logits = model_2l.run_with_hooks(clean_prompt, fwd_hooks=fwd_hooks)[0, -1]
           patching_effect[layer, head] = prediction_logits[clean_answer_index] - prediction_logits[corrupt_answer_index]

    # Plot
    head_labels = [f"(head {i:2})" for i in range(n_heads)]
    layer_labels = [f"(layer {i:2})" for i in range(n_layers)]

    imshow(patching_effect.T, yticks=head_labels, xticks=layer_labels, ylabel="head", xlabel="layer",
           zlabel="Logit difference", title=f"Patching with{corrupt_tokens[4]} /{corrupt_tokens[5]} /{corrupt_tokens[11]}", width=600, height=380)


## Bonus
This time we patch one attention head at one token position to see exactly where each head is useful (basically the same to previous part).

In [22]:
clean_prompt = "Her name was Alex Hart. Tomorrow at lunch time Alex"
#corrupt_prompt = "Her name was Sarah Hart. Tomorrow at lunch time Alex"
#corrupt_prompt = "Her name was Alex Hart. Tomorrow at lunch time Sarah"
corrupt_prompt = "Her name was Alex Carroll. Tomorrow at lunch time Alex"
corrupt_tokens = model_2l.to_str_tokens(corrupt_prompt)
clean_tokens = model_2l.to_str_tokens(clean_prompt)
_, corrupt_cache = model_2l.run_with_cache(corrupt_prompt)

clean_answer_index = model_2l.tokenizer.encode(" Hart")[0]
corrupt_answer_index = model_2l.tokenizer.encode(" Carroll")[0]

n_layers = model_2l.cfg.n_layers
n_heads = model_2l.cfg.n_heads
n_pos = len(clean_tokens)

def patch_head_result(activations, hook, layer=None, head=None, pos=None):
   activations[:, pos, head, :] = corrupt_cache[hook.name][:, pos, head, :]
   return activations

patching_effect = torch.zeros(n_layers*n_heads, n_pos)
for layer in range(n_layers):
    for head in range(n_heads):
      for pos in range(n_pos):
          fwd_hooks = [(f"blocks.{layer}.attn.hook_result", partial(patch_head_result, layer=layer, head=head, pos=pos))]
          prediction_logits = model_2l.run_with_hooks(clean_prompt, fwd_hooks=fwd_hooks)[0, -1]
          patching_effect[n_heads*layer+head, pos] = prediction_logits[clean_answer_index] - prediction_logits[corrupt_answer_index]

In [23]:
# Plot
token_labels = [f"(pos {i:2}) {t}" for i, t in enumerate(clean_tokens)]
layerhead_labels = [f"{l}.{h}" for l in range(n_layers) for h in range(n_heads)]
imshow(patching_effect, xticks=token_labels, yticks=layerhead_labels, xlabel="position", ylabel="layer.head",
           zlabel="Logit difference", title=f"Patching with{corrupt_tokens[4]} /{corrupt_tokens[5]} /{corrupt_tokens[11]}", width=700, height=800)



Same patching loop but it repeats for multiple corrupt prompts.

In [24]:
model_2l.cfg.use_attn_result = True


clean_prompt = "Her name was Alex Hart. Tomorrow at lunch time Alex"
clean_tokens = model_2l.to_str_tokens(clean_prompt)

# Indices of the right and wrong answers (lastnames) to judge what the model predicts
clean_answer_index = model_2l.tokenizer.encode(" Hart")[0]
corrupt_answer_index = model_2l.tokenizer.encode(" Carroll")[0]


n_layers = model_2l.cfg.n_layers
n_heads = model_2l.cfg.n_heads
n_pos = len(clean_tokens)

def patch_head_result(activations, hook, layer=None, head=None, pos=None):
   activations[:, pos, head, :] = corrupt_cache[hook.name][:, pos, head, :]
   return activations


for corrupt_prompt in ["Her name was Alex Carroll. Tomorrow at lunch time Alex",
                       "Her name was Sarah Hart. Tomorrow at lunch time Alex",
                       "Her name was Alex Hart. Tomorrow at lunch time Sarah"]:
    corrupt_tokens = model_2l.to_str_tokens(corrupt_prompt)
    _, corrupt_cache = model_2l.run_with_cache(corrupt_prompt)

    patching_effect = torch.zeros(n_layers*n_heads, n_pos)
    for layer in range(n_layers):
        for head in range(n_heads):
          for pos in range(n_pos):
              fwd_hooks = [(f"blocks.{layer}.attn.hook_result", partial(patch_head_result, layer=layer, head=head, pos=pos))]
              prediction_logits = model_2l.run_with_hooks(clean_prompt, fwd_hooks=fwd_hooks)[0, -1]
              patching_effect[n_heads*layer+head, pos] = prediction_logits[clean_answer_index] - prediction_logits[corrupt_answer_index]
    # Plot
    token_labels = [f"(pos {i:2}) {t}" for i, t in enumerate(clean_tokens)]
    layerhead_labels = [f"{l}.{h}" for l in range(n_layers) for h in range(n_heads)]
    imshow(patching_effect, xticks=token_labels, yticks=layerhead_labels, xlabel="position", ylabel="layer.head",
               zlabel="Logit difference", title=f"Patching with{corrupt_tokens[4]} /{corrupt_tokens[5]} /{corrupt_tokens[11]}", width=500, height=500)



#### Better color plots

Since the baseline logit diff is already a latge number

In [25]:
model_2l.cfg.use_attn_result = True


clean_prompt = "Her name was Alex Hart. Tomorrow at lunch time Alex"
clean_tokens = model_2l.to_str_tokens(clean_prompt)
clean_prediction_logits = model_2l(clean_prompt)[0, -1]
clean_prediction_logit_diff = clean_prediction_logits[clean_answer_index] - clean_prediction_logits[corrupt_answer_index]

# Indices of the right and wrong answers (lastnames) to judge what the model predicts
clean_answer_index = model_2l.tokenizer.encode(" Hart")[0]
corrupt_answer_index = model_2l.tokenizer.encode(" Carroll")[0]


n_layers = model_2l.cfg.n_layers
n_heads = model_2l.cfg.n_heads
n_pos = len(clean_tokens)

def patch_head_result(activations, hook, layer=None, head=None, pos=None):
   activations[:, pos, head, :] = corrupt_cache[hook.name][:, pos, head, :]
   return activations


for corrupt_prompt in ["Her name was Alex Carroll. Tomorrow at lunch time Alex",
                       "Her name was Sarah Hart. Tomorrow at lunch time Alex",
                       "Her name was Alex Hart. Tomorrow at lunch time Sarah"]:
    corrupt_tokens = model_2l.to_str_tokens(corrupt_prompt)
    _, corrupt_cache = model_2l.run_with_cache(corrupt_prompt)

    patching_effect = torch.zeros(n_layers*n_heads, n_pos)
    for layer in range(n_layers):
        for head in range(n_heads):
          for pos in range(n_pos):
              fwd_hooks = [(f"blocks.{layer}.attn.hook_result", partial(patch_head_result, layer=layer, head=head, pos=pos))]
              prediction_logits = model_2l.run_with_hooks(clean_prompt, fwd_hooks=fwd_hooks)[0, -1]
              patching_effect[n_heads*layer+head, pos] = prediction_logits[clean_answer_index] - prediction_logits[corrupt_answer_index]
    # Plot
    token_labels = [f"(pos {i:2}) {t}" for i, t in enumerate(clean_tokens)]
    layerhead_labels = [f"{l}.{h}" for l in range(n_layers) for h in range(n_heads)]
    imshow(patching_effect-clean_prediction_logit_diff, xticks=token_labels, yticks=layerhead_labels, xlabel="position", ylabel="layer.head",
               zlabel="Logit difference difference", title=f"Patching with{corrupt_tokens[4]} /{corrupt_tokens[5]} /{corrupt_tokens[11]}", width=500, height=500)


## Steven's attempt (Bonus)
Basically just change the prompt to see and test how it works.

In [26]:
clean_prompt = "Our team members include Steven. Tommorrow during the meeting one of the members"
corrupt_prompt = "Our team members include Mario. Tommorrow during the meeting one of the members"
corrupt_tokens = model_2l.to_str_tokens(corrupt_prompt)
clean_tokens = model_2l.to_str_tokens(clean_prompt)
_, corrupt_cache = model_2l.run_with_cache(corrupt_prompt)

clean_answer_index = model_2l.tokenizer.encode(" Steven")[0]
corrupt_answer_index = model_2l.tokenizer.encode(" Mario")[0]

n_layers = model_2l.cfg.n_layers
n_heads = model_2l.cfg.n_heads
n_pos = len(clean_tokens)

def patch_head_result(activations, hook, layer=None, head=None, pos=None):
   activations[:, pos, head, :] = corrupt_cache[hook.name][:, pos, head, :]
   return activations

patching_effect = torch.zeros(n_layers*n_heads, n_pos)
for layer in range(n_layers):
    for head in range(n_heads):
      for pos in range(n_pos):
          fwd_hooks = [(f"blocks.{layer}.attn.hook_result", partial(patch_head_result, layer=layer, head=head, pos=pos))]
          prediction_logits = model_2l.run_with_hooks(clean_prompt, fwd_hooks=fwd_hooks)[0, -1]
          patching_effect[n_heads*layer+head, pos] = prediction_logits[clean_answer_index] - prediction_logits[corrupt_answer_index]

The plot shows the changes of the model’s preference for the clean and corrupted continuation, indicating which layer.head pairs might be most influential for the final prediction.


In [27]:
# Plot
token_labels = [f"(pos {i:2}) {t}" for i, t in enumerate(clean_tokens)]
layerhead_labels = [f"{l}.{h}" for l in range(n_layers) for h in range(n_heads)]
imshow(patching_effect, xticks=token_labels, yticks=layerhead_labels, xlabel="position", ylabel="layer.head",
           zlabel="Logit difference", title=f"Patching with{corrupt_tokens[5]} /{corrupt_tokens[15]}", width=700, height=800)



#### Steven's attempt (Better color plots)

Since the baseline logit diff is already a latge number

In [28]:
model_2l.cfg.use_attn_result = True


clean_prompt = "Our team members include Steven. Tommorrow during the meeting one of the members"
clean_tokens = model_2l.to_str_tokens(clean_prompt)

# Indices of the right and wrong answers (lastnames) to judge what the model predicts
clean_answer_index = model_2l.tokenizer.encode(" Steven")[0]
corrupt_answer_index = model_2l.tokenizer.encode(" Mario")[0]

clean_prediction_logits = model_2l(clean_prompt)[0, -1]
clean_prediction_logit_diff = clean_prediction_logits[clean_answer_index] - clean_prediction_logits[corrupt_answer_index]


n_layers = model_2l.cfg.n_layers
n_heads = model_2l.cfg.n_heads
n_pos = len(clean_tokens)

def patch_head_result(activations, hook, layer=None, head=None, pos=None):
   activations[:, pos, head, :] = corrupt_cache[hook.name][:, pos, head, :]
   return activations


for corrupt_prompt in ["Our team members include Mario. Tommorrow during the meeting one of the members"]:
    corrupt_tokens = model_2l.to_str_tokens(corrupt_prompt)
    _, corrupt_cache = model_2l.run_with_cache(corrupt_prompt)

    patching_effect = torch.zeros(n_layers*n_heads, n_pos)
    for layer in range(n_layers):
        for head in range(n_heads):
          for pos in range(n_pos):
              fwd_hooks = [(f"blocks.{layer}.attn.hook_result", partial(patch_head_result, layer=layer, head=head, pos=pos))]
              prediction_logits = model_2l.run_with_hooks(clean_prompt, fwd_hooks=fwd_hooks)[0, -1]
              patching_effect[n_heads*layer+head, pos] = prediction_logits[clean_answer_index] - prediction_logits[corrupt_answer_index]
    # Plot
    token_labels = [f"(pos {i:2}) {t}" for i, t in enumerate(clean_tokens)]
    layerhead_labels = [f"{l}.{h}" for l in range(n_layers) for h in range(n_heads)]
    imshow(patching_effect-clean_prediction_logit_diff, xticks=token_labels, yticks=layerhead_labels, xlabel="position", ylabel="layer.head",
               zlabel="Logit difference difference", title=f"Patching with{corrupt_tokens[5]} /{corrupt_tokens[15]}", width=500, height=500)


# Method 3: Circuitsvis attention plots
This method just shows us which token each attention head is focusing on using a visualization tool, where in the pattern, rows are the current tokens and columns are the tokens they attend to (how strongly each token reads infomation from each earlier token when updating its own representation).

- Attention pattern is a softmax normalized and scaled dot product of query and key matrices.

In [29]:
%pip install circuitsvis


In [30]:
import circuitsvis as cv


In [31]:
prompt = "Her name was Alex Hart. Tomorrow at lunch time Alex"
_, cache = model_2l.run_with_cache(prompt)
cv.attention.attention_patterns(tokens=model_2l.to_str_tokens(prompt),
                                attention=cache[f'blocks.0.attn.hook_pattern'][0])

In [32]:
import circuitsvis as cv
layer = 1
cv.attention.attention_patterns(tokens=model_2l.to_str_tokens("Her name was Alex Hart. Tomorrow at lunch time Alex"),
                                attention=cache[f'blocks.{layer}.attn.hook_pattern'][0])

This is again my attempt to try out different prompts, which do not seem to be as effective/useful as the given example.

In [34]:
prompt = "Our team members include Steven. Tommorrow during the meeting one of the members"
_, cache = model_2l.run_with_cache(prompt)
cv.attention.attention_patterns(tokens=model_2l.to_str_tokens(prompt),
                                attention=cache[f'blocks.0.attn.hook_pattern'][0])